In [1]:
import os
from dotenv import load_dotenv

from src.config import LLMConf, EmbedderConf, KnowledgeGraphConfig
from src.factory.embeddings import get_embeddings
from src.graph.knowledge_graph import KnowledgeGraph
from neo4j import Query, Session
from src.schema import Chunk
from src.agents.graph_qa import GraphAgentResponder
env = load_dotenv('config.env')

from pgs.utils import get_configuration_from_env

from src.utils.logger import get_logger
logger = get_logger(__name__)

conf = get_configuration_from_env()

2025-04-06 20:54:33.327 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-04-06 20:54:33.329 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-04-06 20:54:33.329 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 20:54:33.419 
  command:

    streamlit run /Users/dylan.tartarini/Desktop/Dylan/knowledge-graphs/.venv/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-04-06 20:54:33.419 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 20:54:33.419 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 20:54:33.421 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-

In [2]:
kg_config = KnowledgeGraphConfig(
    uri=os.getenv("NEO4J_URI"),
    user=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    index_name='vector'
)

embedder_conf = EmbedderConf(
    type=os.getenv("EMBEDDINGS_TYPE"),
    model=os.getenv("EMBEDDINGS_MODEL_NAME"),
)

model_conf=LLMConf(
    type=os.getenv("RE_MODEL_TYPE"),
    model=os.getenv("RE_MODEL_NAME"), 
    temperature=os.getenv("RE_MODEL_TEMPERATURE"), 
    deployment=os.getenv("RE_MODEL_DEPLOYMENT"),
    api_key=os.getenv("RE_API_KEY"),
    endpoint=os.getenv("RE_MODEL_ENDPOINT"),
    api_version=os.getenv("RE_MODEL_API_VERSION") or None
)

embeddings = get_embeddings(conf=embedder_conf)

kg = KnowledgeGraph(conf=kg_config,embeddings_model=embeddings)

## Cypher Chains
Here we use a method to answer using  only Cypher queries

In [3]:
responder = GraphAgentResponder(
    qa_llm_conf=conf.qa_model,
    cypher_llm_conf=conf.qa_model,
    graph=kg,
    # rephrase_llm_conf=conf.qa_model
)

2025-04-06 18:50:47,289 - src.factory.llm - INFO - Fetching LLM model 'gemma2-9b-it'..
2025-04-06 18:50:47,460 - src.factory.llm - INFO - Initialized LLM of type: 'ModelType.GROQ'
2025-04-06 18:50:47,461 - src.factory.llm - INFO - Fetching LLM model 'gemma2-9b-it'..
2025-04-06 18:50:47,494 - src.factory.llm - INFO - Initialized LLM of type: 'ModelType.GROQ'


In [4]:
responder.answer_with_cypher(query="Who works for Europe Direct?")

'Eva Hrncirova and Quentin Cortes work for Europe Direct. \n'

In [5]:
responder.answer_with_cypher(
    query="What legislation has been discussed that regards Apple?", 
    intermediate_steps=True
)

('The Digital Markets Act has been discussed regarding Apple. \n',
 [{'query': "MATCH (c:Chunk)-[:MENTIONS]->(l:Legislation) WHERE c.text CONTAINS 'Apple' RETURN l.id, l.name\n"},
  {'context': [{'l.id': 'Digital Markets Act',
     'l.name': 'Digital Markets Act'},
    {'l.id': 'Digital Markets Act', 'l.name': 'Digital Markets Act'}]}])

In [ ]:
# rephraser not working as expected
responder_with_rephrase = GraphAgentResponder(
    qa_llm_conf=conf.qa_model,
    cypher_llm_conf=conf.qa_model,
    graph=kg,
    rephrase_llm_conf=conf.qa_model
)

2025-04-06 18:53:02,260 - src.factory.llm - INFO - Fetching LLM model 'gemma2-9b-it'..
2025-04-06 18:53:02,356 - src.factory.llm - INFO - Initialized LLM of type: 'ModelType.GROQ'
2025-04-06 18:53:02,357 - src.factory.llm - INFO - Fetching LLM model 'gemma2-9b-it'..
2025-04-06 18:53:02,389 - src.factory.llm - INFO - Initialized LLM of type: 'ModelType.GROQ'
2025-04-06 18:53:02,390 - src.factory.llm - INFO - Fetching LLM model 'gemma2-9b-it'..
2025-04-06 18:53:02,423 - src.factory.llm - INFO - Initialized LLM of type: 'ModelType.GROQ'


In [7]:
responder_with_rephrase.answer_with_cypher(query="Who works for Europe Direct?")

2025-04-06 18:53:07,898 - src.agents.graph_qa - INFO - Rephrased Question: FIND Person WHERE Person.WORKS_FOR = 'Europe Direct' 

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: WORKS_FOR)} {position: line: 1, column: 26, offset: 25} for query: "MATCH (p:Person) WHERE p.WORKS_FOR = 'Europe Direct' RETURN p\n"


"I don't know the answer. \n"

In [8]:
responder_with_rephrase.answer_with_cypher(
    query="What legislation has been discussed that regards Apple?", 
    intermediate_steps=True
)

2025-04-06 18:53:42,630 - src.agents.graph_qa - INFO - Rephrased Question: FIND  Legislation  MENTIONS 'Apple'  



("I don't know the answer. \n",
 [{'query': "MATCH (l:Legislation)<-[:MENTIONS]-(c:Chunk) WHERE c.text CONTAINS 'Apple' RETURN l\n"},
  {'context': [{'l': {'community_leiden': 2,
      'closeness': 0.07395602614661552,
      'name': 'Digital Markets Act',
      'id': 'Digital Markets Act',
      'betweenness': 0.013308240354886411,
      'pagerank': 0.005754031618527281,
      'community_louvain': 1}},
    {'l': {'community_leiden': 2,
      'closeness': 0.07395602614661552,
      'name': 'Digital Markets Act',
      'id': 'Digital Markets Act',
      'betweenness': 0.013308240354886411,
      'pagerank': 0.005754031618527281,
      'community_louvain': 1}}]}])

### Vanilla RAG
Uses only vanilla RAG to answer the user's question.  
If `use_adjacent_chunks=True` will query the graph for additional context   
compared to the Chunks retrieved by the similarity search. Latency will be higher due to expanded context. 

In [4]:
responder = GraphAgentResponder(
    qa_llm_conf=conf.qa_model,
    cypher_llm_conf=conf.qa_model,
    graph=kg
    # rephrase_llm_conf=conf.qa_model
)

2025-04-06 20:07:29,368 - src.factory.llm - INFO - Fetching LLM model 'gemma2-9b-it'..
2025-04-06 20:07:29,514 - src.factory.llm - INFO - Initialized LLM of type: 'ModelType.GROQ'
2025-04-06 20:07:29,514 - src.factory.llm - INFO - Fetching LLM model 'gemma2-9b-it'..
2025-04-06 20:07:29,547 - src.factory.llm - INFO - Initialized LLM of type: 'ModelType.GROQ'


In [4]:
responder.answer_with_context("Who works for Europe Direct?")

"I don't know.  \n"

In [5]:
responder.answer_with_context(
    query="Who works for Europe Direct?", 
    use_adjacent_chunks=True
)

'The provided text does not say who works for Europe Direct. \n'

In [6]:
responder.answer_with_context(
    query="What legislation has been discussed that regards Apple?"
)

'The legislation discussed that regards Apple is the Digital Markets Act (DMA).  \n'

In [7]:
responder.answer_with_context(
    query="What legislation has been discussed that regards Apple?",
    use_adjacent_chunks=True
)

'The legislation discussed that regards Apple is the Digital Markets Act (DMA).  \n'

In [8]:
responder.answer_with_context("Who is Eva Hrncirova?")

'Eva Hrncirova is a press contact for Glenn Micallef, Commissioner for Intergenerational Fairness, Youth, Culture and Sport.  \n'

In [5]:
responder.answer_with_context("Who is Eva Hrncirova?", use_adjacent_chunks=True)

'Eva Hrncirova is a press contact for Discover EU.  \n'

In [6]:
responder.answer_with_context("Who is the Commissioner for Intergenerational Fairness?")

"I don't know. \n"

In [9]:
responder.answer_with_context(
    query="Who is the Commissioner for Intergenerational Fairness?",
    use_adjacent_chunks=True
)

'The provided text does not contain the answer to your question. \n'

In [8]:
responder.answer_with_context(query="Who is Glenn Micallef?")

"I don't know.  \n"

In [7]:
responder.answer_with_context(query="Who is Glenn Micallef?", use_adjacent_chunks=True)

'Glenn Micallef is the Commissioner for Intergenerational Fairness, Youth, Culture and Sport.  \n'

In [10]:
responder.answer_with_context("what is the Digital Markets Act?")

'The Digital Markets Act (DMA) aims to ensure contestable and fair markets in the digital sector. It regulates gatekeepers, which are large digital platforms that provide an important gateway between business users and consumers, whose position can grant them the power to create a bottleneck in the digital economy.  \n'

In [11]:
responder.answer_with_context("what is the Digital Markets Act?", use_adjacent_chunks=True)

'The Digital Markets Act (DMA) aims to ensure contestable and fair markets in the digital sector. It regulates gatekeepers, which are large digital platforms that provide an important gateway between business users and consumers, whose position can grant them the power to create a bottleneck in the digital economy.  \n'

## Community Reports

In [3]:
communities = kg.get_communities()
communities

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: " \n            MATCH (n)-[r]-(m)  \n            WHERE n.community_leiden IS NOT NULL\n            OPTIONAL MATCH (chunk:Chunk) WHERE chunk.community_leiden = n.community_leiden  \n            WITH \n                'leiden' AS community_type, \n                n.community_leiden AS community_id, \n                count(DISTINCT n) AS community_size, \n                collect(DISTINCT elementId(n)) AS entity_ids,\n                collect(DISTINCT n.name) AS names,\n                collect(DISTINCT elementId(r)) AS relationship_ids,  \n                collect(DISTINCT type(r)) AS relationship_types,\n                collect(DISTINCT elementId(chunk)) AS chunk_ids    \n            RETUR

[Community(community_type='leiden', community_id=0, community_size=34, entity_ids=['4:44890bb9-4945-4a7c-8f87-381effaa2643:65', '4:44890bb9-4945-4a7c-8f87-381effaa2643:66', '4:44890bb9-4945-4a7c-8f87-381effaa2643:68', '4:44890bb9-4945-4a7c-8f87-381effaa2643:69', '4:44890bb9-4945-4a7c-8f87-381effaa2643:71', '4:44890bb9-4945-4a7c-8f87-381effaa2643:78', '4:44890bb9-4945-4a7c-8f87-381effaa2643:79', '4:44890bb9-4945-4a7c-8f87-381effaa2643:80', '4:44890bb9-4945-4a7c-8f87-381effaa2643:81', '4:44890bb9-4945-4a7c-8f87-381effaa2643:82', '4:44890bb9-4945-4a7c-8f87-381effaa2643:90', '4:44890bb9-4945-4a7c-8f87-381effaa2643:91', '4:44890bb9-4945-4a7c-8f87-381effaa2643:92', '4:44890bb9-4945-4a7c-8f87-381effaa2643:135', '4:44890bb9-4945-4a7c-8f87-381effaa2643:136', '4:44890bb9-4945-4a7c-8f87-381effaa2643:137', '4:44890bb9-4945-4a7c-8f87-381effaa2643:138', '4:44890bb9-4945-4a7c-8f87-381effaa2643:139', '4:44890bb9-4945-4a7c-8f87-381effaa2643:140', '4:44890bb9-4945-4a7c-8f87-381effaa2643:141', '4:44890bb

In [4]:
communities[0].chunks

[Chunk(chunk_id='4:44890bb9-4945-4a7c-8f87-381effaa2643:78', text='short quiz about the EU. Passes will be awarded to the top-ranked applicants until all tickets are distributed. Travellers can design their own European journey or choose from suggested routes. They may follow the New European Bauhaus Route, showcasing vibrant, sustainable cities, or choose the Green Route, taking them takes to some of the most sustainable cities and nature-friendly destinations across the continent. This route features award-winning Green Capitals and Green Leaf cities, stunning parks, and nature reserves. Along the way, they can use a discount card packed with thousands of offers on transport, museums, food, accommodation, sports and more across 36 countries. Discover EU will make this experience accessible to all. Special support is available for participants with disabilities, health conditions or fewer opportunities. Additional arrangements are also offered to those living in remote areas, islands 

In [5]:
responder = GraphAgentResponder(
    qa_llm_conf=conf.qa_model,
    cypher_llm_conf=conf.qa_model,
    graph=kg
    # rephrase_llm_conf=conf.qa_model
)

2025-04-06 20:55:34,005 - src.factory.llm - INFO - Fetching LLM model 'gemma2-9b-it'..
2025-04-06 20:55:34,153 - src.factory.llm - INFO - Initialized LLM of type: 'ModelType.GROQ'
2025-04-06 20:55:34,154 - src.factory.llm - INFO - Fetching LLM model 'gemma2-9b-it'..
2025-04-06 20:55:34,187 - src.factory.llm - INFO - Initialized LLM of type: 'ModelType.GROQ'


In [6]:
responder.answer_with_community_reports(query="what is the Digital Markets Act?")

[(Document(metadata={'community_leiden': 13, 'closeness': 0.0, 'pagerank': 0.0035649165962808836, 'betweenness': 0.0, 'community_louvain': 14, 'community_type': 'leiden', 'community_id': 4, 'community_size': 11}, page_content="The European Commission issued two decisions requiring Apple to comply with the Digital Markets Act's interoperability obligation. These decisions specify measures for Apple to enable interoperability with iOS for third-party connected devices, aiming to increase consumer choice and foster innovation in the European market.  \n"),
  0.8276760578155518),
 (Document(metadata={'community_leiden': 10, 'closeness': 0.0, 'pagerank': 0.0035649165962808836, 'betweenness': 0.0, 'community_louvain': 11, 'community_type': 'leiden', 'community_id': 1, 'community_size': 21}, page_content='The European Commission has issued legally binding decisions requiring Apple to improve interoperability between its iOS and iPadOS operating systems and third-party apps and hardware.  Thes

In [7]:
responder.answer_with_community_reports(query="What is the European Commission telling Apple to do?")

[(Document(metadata={'community_leiden': 13, 'closeness': 0.0, 'pagerank': 0.0035649165962808836, 'betweenness': 0.0, 'community_louvain': 14, 'community_type': 'leiden', 'community_id': 4, 'community_size': 11}, page_content="The European Commission issued two decisions requiring Apple to comply with the Digital Markets Act's interoperability obligation. These decisions specify measures for Apple to enable interoperability with iOS for third-party connected devices, aiming to increase consumer choice and foster innovation in the European market.  \n"),
  0.8790302872657776),
 (Document(metadata={'community_leiden': 10, 'closeness': 0.0, 'pagerank': 0.0035649165962808836, 'betweenness': 0.0, 'community_louvain': 11, 'community_type': 'leiden', 'community_id': 1, 'community_size': 21}, page_content='The European Commission has issued legally binding decisions requiring Apple to improve interoperability between its iOS and iPadOS operating systems and third-party apps and hardware.  Thes

## Full Answer

In [ ]:
responder.answer(query="What is Europe doing for remote areas?", filter={"community_leiden": 0})